In [15]:
from deap import base, creator, tools, algorithms
import random
import numpy as np
import pandas as pd

In [16]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('Promedio', np.mean)
stats.register('Desviación Estándar', np.std)
stats.register('Mínimo', np.min)
stats.register('Máximo', np.max)

In [37]:
# Número de áreas operativas
n_areas = 4

# Para cada área, lista de recursos disponibles
# Cada recurso es una tupla: (costo, calidad)

# resources[area][opcion] = (costo, calidad)
resources = [
    [(1000, 5), (1500, 7), (1800, 9)],   # Área 1
    [(800, 4), (1200, 6), (1600, 8)],    # Área 2
    [(900, 3), (1300, 6), (2000, 10)],   # Área 3
    [(700, 2), (1100, 5), (1700, 7)]     # Área 4
]

data = pd.DataFrame([
    {'area': 'A1', 'opcion': 0, 'costo': 1000, 'calidad': 5},
    {'area': 'A1', 'opcion': 1, 'costo': 1500, 'calidad': 7},
    {'area': 'A1', 'opcion': 2, 'costo': 1800, 'calidad': 9},
    {'area': 'A2', 'opcion': 0, 'costo': 800,  'calidad': 4},
    {'area': 'A2', 'opcion': 1, 'costo': 1200, 'calidad': 6},
    {'area': 'A2', 'opcion': 2, 'costo': 1600, 'calidad': 8},
    {'area': 'A3', 'opcion': 0, 'costo': 900,  'calidad': 3},
    {'area': 'A3', 'opcion': 1, 'costo': 1300, 'calidad': 6},
    {'area': 'A3', 'opcion': 2, 'costo': 2000, 'calidad': 10},
    {'area': 'A4', 'opcion': 0, 'costo': 700,  'calidad': 2},
    {'area': 'A4', 'opcion': 1, 'costo': 1100, 'calidad': 5},
    {'area': 'A4', 'opcion': 2, 'costo': 1700, 'calidad': 7}
])

resources = []
for area in data['area'].unique():
    recursos_area = data[data['area'] == area][['costo', 'calidad']].values.tolist()
    resources.append(recursos_area)


In [38]:
def evaluar_individuo(individuo, recursos, presupuesto, calidad_minima):
    costo_total = 0
    calidad_total = 0

    for i, opcion in enumerate(individuo):
        costo, calidad = recursos[i][opcion]
        costo_total += costo
        calidad_total += calidad

    # Penalizar si no cumple restricciones
    if costo_total > presupuesto or calidad_total < calidad_minima:
        return 0, 999999  # ✅ Cero calidad y costo muy alto = penalización

    # Optimización multiobjetivo: (calidad ↑, costo ↓)
    return calidad_total, costo_total



In [54]:
presupuesto = 8000
calidad_minima = 40

# Crear tipos
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))  # Calidad ↑, Costo ↓
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Toolbox
toolbox = base.Toolbox()
toolbox.register("attribute", random.randint, 0, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=4)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluador
toolbox.register("evaluate", evaluar_individuo, recursos=resources,
                 presupuesto=presupuesto, calidad_minima=calidad_minima)

toolbox.register("select", tools.selNSGA2)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=2, indpb=0.2)

In [55]:
population = toolbox.population(n=1000)
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=800, stats = stats, verbose=True)


gen	nevals	Promedio	Desviación Estándar	Mínimo	Máximo
0  	1000  	500000  	500000             	0     	999999
1  	766   	500000  	500000             	0     	999999
2  	779   	500000  	500000             	0     	999999
3  	747   	500000  	500000             	0     	999999
4  	762   	500000  	500000             	0     	999999
5  	761   	500000  	500000             	0     	999999
6  	756   	500000  	500000             	0     	999999
7  	783   	500000  	500000             	0     	999999
8  	729   	500000  	500000             	0     	999999
9  	755   	500000  	500000             	0     	999999
10 	759   	500000  	500000             	0     	999999
11 	784   	500000  	500000             	0     	999999
12 	738   	500000  	500000             	0     	999999
13 	762   	500000  	500000             	0     	999999
14 	775   	500000  	500000             	0     	999999
15 	751   	500000  	500000             	0     	999999
16 	761   	500000  	500000             	0     	999999
17 	784   	500000  	500000  

In [56]:
# Filtrar individuos válidos
validos = [ind for ind in population if evaluar_individuo(ind, resources, presupuesto, calidad_minima)[1] < 999999]

print(f"Número de soluciones válidas: {len(validos)}")

if validos:
    # Ahora sí: el mejor entre los válidos
    mejor_valido = tools.selBest(validos, 1)[0]
    print(f"✅ Mejor combinación válida: {mejor_valido}")
    print(f"📊 Calidad y costo: {evaluar_individuo(mejor_valido, resources, presupuesto, calidad_minima)}")
else:
    print("❌ No se encontraron soluciones válidas.")



Número de soluciones válidas: 0
❌ No se encontraron soluciones válidas.
